In [165]:
import Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using Makie
using WGLMakie
using ForwardDiff
WGLMakie.activate!()

 Activating environment at `~/.julia/dev/OptiViz/Project.toml`


## Root-finding using Newton's Method

In [166]:
f(x) = (x-1)*(x-3)*(x-5)*x
∇f(x) = ForwardDiff.derivative(f,x)
∇²f(x) = ForwardDiff.derivative(x_->ForwardDiff.derivative(f,x_),x)

∇²f (generic function with 1 method)

In [167]:
N = 101
x = range(-0.5,5.5,length=N)
s = lines(x, f.(x))
xlims!(s, x[1], x[end])
ylims!(s, -12, 30)

# Initial Point
x1 = Node(3.8)
y1 = @lift f($x1)
p1 = @lift Point($x1, $y1)
scatter!(s, p1, color=:blue)

# Plot Tangent Line
xt = @lift range($x1-10,$x1+10,length=3)
yt = @lift $y1 .+ ∇f($x1)*($xt .- $x1)
lines!(s, xt, yt, color=:blue)

# Newton Step
x2 = @lift $x1 - $y1/∇f($x1)
y2 = @lift f($x2)
p2 = @lift Point($x2, $y2)

# Error
err = @lift [Point($x2,0), $p2]
lines!(s, err, color=:red)
scatter!(s, p2, color=:green)

In [59]:
# Initial Guess
x1[] = 3.99;

In [67]:
# Newton's method
x = x1[]                # get current value of x
dx = -f(x)/∇f(x)        # Compute new value using Newton's Method
x1[] = x + dx           # set new value of x
println("Step Size: ", abs(dx))
println("Error: ", abs(f(x+dx)))
println("New x: ", x+dx)

Step Size: 0.0
Error: 0.0
New x: 3.0


## Minimization using Newton's Method

In [68]:
# Plot function
N = 101
xs = range(-0.5,5.5,length=N)
s = lines(xs, f.(xs))
xlims!(s, xs[1], xs[end])
ylims!(s, -12, 30)

# Initial point
x1[] = 5.2
scatter!(s, p1, color=:blue)

# Create a quadratic approximation
m(x,x0) = f(x0) + ∇f(x0)*(x - x0) + 0.5*∇²f(x0)*(x-x0)^2
xq = @lift range($x1-5,$x1+5,length=100)
yq = @lift m.($xq,$x1)
lines!(s, xq, yq, color=:blue)

# Take Newton step
δx = @lift -∇f($x1)/∇²f($x1)
x2 = @lift $x1 + $δx
y2 = @lift f($x2)
p2 = @lift Point($x2, $y2)
scatter!(s, p2, color=:green)

# Plot the error
err2 = @lift [Point($x2, m($x2,$x1)), $p2]
lines!(s, err2, color=:red)

In [73]:
# Newton's Method
x = x1[]                # Get current value
dx = -∇f(x) / ∇²f(x)    # Compute step direction
x_new = x + dx          # New iterate
x1[] = x + dx           # save new step

println("Gradient Norm: ", norm(∇f(x1[])))
println("Model Error: ", abs(m(x_new, x) - f(x_new)))
println("New x: ", x_new)
println("Value: ", f(x_new))

Gradient Norm: 1.6813439529528296e-10
Model Error: 0.0
New x: 4.253749245867311
Value: -12.949453288874253


In [74]:
scene, layout = layoutscene(30, resolution=(800,800))
ax1 = layout[1,1] = LAxis(scene, title = "Function")
ax2 = layout[2,1] = LAxis(scene, title = "Gradient")

lines!(ax1, xs, f.(xs))
lines!(ax2, xs, ∇f.(xs))
hlines!(ax2, [0], color=:gray)

# Initial point
x1[] = 5.2
scatter!(ax1, p1, color=:blue)

# Create a quadratic approximation
m(x,x0) = f(x0) + ∇f(x0)*(x - x0) + 0.5*∇²f(x0)*(x-x0)^2
xq = @lift range($x1-5,$x1+5,length=100)
yq = @lift m.($xq,$x1)
lines!(ax1, xq, yq, color=:blue)

# Take Newton step
δx = @lift -∇f($x1)/∇²f($x1)
x2 = @lift $x1 + $δx
y2 = @lift f($x2)
p2 = @lift Point($x2, $y2)
scatter!(ax1, p2, color=:green)

# Plot the error
err2 = @lift [Point($x2, m($x2,$x1)), $p2]
lines!(ax1, err2, color=:red)

# Initial Point
z1 = @lift ∇f($x1)
d1 = @lift Point($x1, $z1)
scatter!(ax2, d1, color=:blue)

# Plot Tangent Line
xt = @lift range($x1-10,$x1+10,length=3)
yt = @lift $z1 .+ ∇²f($x1)*($xt .- $x1)
lines!(ax2, xt, yt, color=:blue)

# Newton Step
z2 = @lift ∇f($x2)
d2 = @lift Point($x2, $z2)

# Error
err = @lift [Point($x2,0), $d2]
lines!(ax2, err, color=:red)
scatter!(ax2, d2, color=:green)

ylims!(ax1, -14, 35)
ylims!(ax2, -50, 90)
xlims!(ax1,xs[1],xs[end])
xlims!(ax2,xs[1],xs[end])
scene

In [81]:
# Newton's Method
x = x1[]                # Get current value
dx = -∇f(x) / ∇²f(x)    # Compute step direction
x_new = x + dx          # New iterate
x1[] = x + dx           # save new step

println("Gradient Norm: ", norm(∇f(x1[])))
println("Model Error: ", abs(m(x_new, x) - f(x_new)))
println("New x: ", x_new)
println("Value: ", f(x_new))

Gradient Norm: 9.325873406851315e-15
Model Error: 0.0
New x: 4.253749245862282
Value: -12.949453288874254


## Line Search

In [164]:
# Try Changing these
c1 = 0.1   # higher is more strict  (0 < c1 < 0.5)
c2 = 0.9    # lower is more strict   (c1 < c2 < 1)

# Create line search function
x = 5.2
dx = -∇f(x) / ∇²f(x)
ϕ(α) = f(x + α*dx)
ϕ′(α) = ∇f(x + α*dx)*dx
α = range(0,10,length=101)
phi = ϕ.(α)

# Plot Wolfe Conditions
suff = ϕ(0.0) .+ c1 .* α * ϕ′(0.0)                # Line for sufficient decrease (Armijo)
gold = ϕ(0.0) .+ α * (1-c1)*ϕ′(0.0)               # Line for Goldstein curvature
curv = fill(NaN,length(α))
curv_strong = fill(NaN,length(α))
good = fill(NaN,length(α))
# gold = fill(NaN,length(α))
igold = phi .>= ϕ(0) .+ (1-c1) * α * ϕ′(0)        # goldstein curvature
icurv = ϕ′.(α) .> c2 * ϕ′(0)                      # curvature condition
icurv_strong = abs.(ϕ′.(α)) .< c2 * abs(ϕ′(0))    # strong curvature condition
isuff = phi .< suff                               # sufficient decrease condition (i.e. Armijo)
igood = icurv_strong .& isuff                     # both satisfied
# gold[igold] .= phi[igold]
curv[icurv] .= phi[icurv]
curv_strong[icurv_strong] .= phi[icurv_strong]
good[igood] .= phi[igood]

# Generate Plot
s,layout = layoutscene()
ax = layout[1,1] = LAxis(s, title = "Line Search Conditions")
a = lines!(ax, α, phi)
b = lines!(ax, α, suff, color=:orange, linewidth=1)
c = lines!(ax, α, curv, color=:cyan, linewidth=2)
d = lines!(ax, α, curv_strong, color=:blue, linewidth=3)
e = lines!(ax, α, good, color=:green, linewidth=6)
g = lines!(ax, α, gold, color=:red, linewidth=1)
leg = LLegend(s, [a,b,c,d,e,g], 
    ["ϕ","Sufficient decrease","Curvature condition","Curvature condition (strong)", 
        "Strong Wolfe","Goldstein curvature"])

layout[1,2] = leg
ax.xlabel = "step size (α)"
ax.ylabel = "ϕ(α)"
ylims!(ax,[-15,30])
s